<a href="https://colab.research.google.com/github/cglima/ftc_dados_python/blob/main/exercicios_fim_ciclo05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercícios de 1/2 Ciclo 05

In [7]:
from google import colab as cl
import pandas as pd
import re
import plotly.express as px

In [4]:
url = 'https://raw.githubusercontent.com/cglima/ftc_dados_python/main/dataset/train.csv'
df = pd.read_csv(url)
df.head()


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


## Limpeza de dados

In [5]:
df1 = df.copy()
# excluir as linhas com os registros vazios (NaN)
# idade dos entregadores vazia
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# ID vazio
linhas_selecionadas = df1['ID'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# City vazia
linhas_selecionadas = df1['City'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# Road_traffic_density vazia
linhas_selecionadas = df1['Road_traffic_density'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# 1. convertendo a coluna Age de texto para numero
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# 2. convertendo a coluna Rating de texto para numero decimal (float)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

# 3. convertendo a coluna order_date de texto para data
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')

# 4. convertendo multiple_deliveries de texto para numero inteiro (int)
linhas_selecionadas = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

# 5. removendo os espaços dentro de strings/texto/object
# df1.iloc[0, 1]
df1.loc[0, 'City']

df1.loc[:, 'ID'] = df1.loc[:, 'ID'].str.strip()
df1.loc[:, 'Delivery_person_ID'] = df1.loc[:, 'Delivery_person_ID'].str.strip()
df1.loc[:, 'Road_traffic_density'] = df1.loc[:, 'Road_traffic_density'].str.strip()
df1.loc[:, 'Type_of_order'] = df1.loc[:, 'Type_of_order'].str.strip()
df1.loc[:, 'Type_of_vehicle'] = df1.loc[:, 'Type_of_vehicle'].str.strip()
df1.loc[:, 'City'] = df1.loc[:, 'City'].str.strip()
df1.loc[:, 'Festival'] = df1.loc[:, 'Festival'].str.strip()

# 6. removendo o texto de números
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply(lambda x: re.findall( r'\d+', x))

# 7. resetando o index do dataset
df1 = df1.reset_index(drop=True)

# 1. Desenhar um gráfico de pizza com a média de idade dos entregadores por cidade

In [12]:
cols = ['Delivery_person_Age', 'City']
df_aux =df1.loc[:, cols].groupby(['City']).mean().reset_index()

px.pie(df_aux, values='Delivery_person_Age', names='City')

 # 2. Desenhe um gráfico de linha, mostrando o número total de entregas diárias feitas por densidade de tráfego igual a “Low” e “Jam”

In [13]:
df_aux = df1.loc[(df1['Road_traffic_density'] == 'Low') | (df1['Road_traffic_density'] == 'Jam'),
                ['Order_Date', 'ID']].groupby('Order_Date').count().reset_index()
df_aux.columns = ['Order_Date', 'Numbers_of_deliveries']

px.line(df_aux, x = 'Order_Date', y = 'Numbers_of_deliveries')

 # 3. Desenhe um gráfico de barras, mostrando as avaliações médias das entregas por semana.

In [14]:
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')

df_deliveries_by_week = df1.loc[:, ['Delivery_person_Ratings', 'week_of_year']].groupby(['week_of_year']).mean().reset_index()

px.bar(df_deliveries_by_week, x = 'week_of_year', y = 'Delivery_person_Ratings')

# 4. Desenhe um gráfico de pizza, mostrando a média de avaliações das entregas feitas por condições climáticas?

In [15]:
df_ratings_by_weather = df1.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']].groupby(['Weatherconditions']).mean().reset_index()

# Limpando os NaN
df_ratings_by_weather = df_ratings_by_weather.loc[df_ratings_by_weather['Weatherconditions'] != 'conditions NaN', :]

px.pie(df_ratings_by_weather, values='Delivery_person_Ratings', names='Weatherconditions')